In [161]:
import numpy as np

Factorising n we look for polynomial P and number m such that P(m) = 0 mod N. This approach takes the numbers close to a*N for integer a and looks for numbers that are a power of some integer. We aim to look for polynomials of degree 3 or 4.

In [162]:
def check_pow_34(a):  #  returns r for which q^r = a for some q if r is 3, 4 or 5
    for k in range(3, 4):
        i = 1
        while i ** k < a:
            i = i + 1
        if i ** k == a:
            return [i, k]
    return 0

In [163]:
def init_poly(n):
    for a in range(2, int(np.floor(n/2))):
        for t in range(a * n - 10, a * n + 11):
            if check_pow_34(t) != 0:
                return [check_pow_34(t)[0], check_pow_34(t)[1], a * n - t]  #  returns [m, degree of polynomial, free term of polynomial]
    return 0

In [164]:
print(init_poly(441))
print(init_poly(2 ** 14 - 1))
print(init_poly(89*113))

[11, 3, -8]
[32, 3, -2]
[194, 3, -2]


Helpful functions: Legendre symbol, sieve prime test, set B of prime factors p with with n quadratic residue mod p and checking if a number is B-smooth, returning its prime powers from the B set.

In [165]:
def gcd(a, b):
    while b:
        a, b = b, a % b
    return abs(a)

def legendre(n, p): # legendre symbol (n/p)
    if n % p == 0:
        return 0
    if n > p:
        n = n % p
    if n ** (int((p - 1) / 2)) % p == 1:
        return 1
    else:
        return -1


def smsieve(n):  # prime check with classic sieving
    if n in [1, 2]:
        return 1
    for i in range(2, int(np.floor(np.sqrt(n))) + 1):
        if n % i == 0:
            return 0
    return 1

def Bset(k, n):  # construct the set of the first k primes p that have (n/p) = +1 together with -1
    B = [-1]
    p = 2
    while k:
        while legendre(n, p) != 1 or smsieve(p) != 1:
            p += 1
        B.append(p)
        p += 1
        k = k - 1
    return B

def Bfactors(n, B):  # for number n and set of primes B with -1 returns the factorisation and powers of n if n is
                     # B-smooth and 0 if n is not B-smooth
    if n < 0:
        basepowers = [1]
        n = int(n * -1)
    else:
        basepowers = [0]
    B = B[1:]
    for prime in B:
        prime_power = 0
        while n % prime == 0:
            n = int(n / prime)
            prime_power += 1
        basepowers.append(prime_power)
    if n == 1:
        return basepowers
    else:
        return 0

In [166]:
def abm_table(m, B, limit):
    table = []
    for a in range(-1 * limit, limit + 1):
        for b in range(-1 * limit, limit + 1):
            if a - b * m != 0 and Bfactors(a - b * m, B) != 0:
                table.append([a, b, a - b * m, Bfactors(a - b * m, B)])
    return table


Define norm in field with roots of polynomial x^3 + t and x^4 + t

In [167]:
def N(a, b, t, deg):
    if deg == 3:
        return a ** 3 + t * b ** 3 - 3 * a * b ** 2
    if deg == 4:
        return a ** 4 + t * b ** 4 + 2 * a ** 2 * b ** 2

Add the factorisation of the norm

In [168]:
def add_algebraic(table, deg, t, B):
    final_table = []
    for row in table:
        alg = N(row[0], row[1], t, deg)
        if Bfactors(alg, B) != 0:
            final_table.append([row[2], alg, Bfactors(alg, B)])
    return final_table

def split_table(table):   #  split to calculate u and v
    us = []
    vs = []
    fac = []
    for row in table:
        us.append(row[0])
        vs.append(row[1])
        fac.append(row[2])
    return [us, vs, fac]

Functions to help us calculate products u and v and determine which rows are linearly dependent mod 2

In [169]:
def partitions(l, d):
    part = []
    d = f"{d:0{len(l)}b}"
    for i in range(len(d)):
        if d[i] == "1":
            part.append(l[i])
    return part

def mult(A, d):
    mult = 1
    d = f"{d:0{len(A)}b}"
    for i in range(len(d)):
        if d[i] == "1":
            mult = mult * A[i]
    return mult

def testifeven(l, B):  #  l is list of lists, B is set of primes with -1
    for i in range(len(B)):
        sumcol = 0
        for j in range(len(l)):
            sumcol += l[j][i]
        if sumcol % 2 != 0:
            return 0
    return 1

In [170]:
def sieve(us, vs, fac, B, n):
    for d in range(2 ** len(us)):
        if len(partitions(fac, d)) >= 2 and testifeven(partitions(fac, d), B) == 1:
            u = mult(us, d)
            v = mult(vs, d)
            if gcd(u - v, n) > 1:
                return gcd(u - v, n)
            if gcd(u + v, n) > 1:
                return gcd(u + v, n)
    return 0

In [171]:
def NFS(n, limit, k):
    pol = init_poly(n)
    m = pol[0]
    deg = pol[1]
    t = pol[2]
    B = Bset(k, n)
    table = abm_table(m, B, limit)
    table = add_algebraic(table, deg, t, B)
    us = split_table(table)[0]
    vs = split_table(table)[1]
    fac = split_table(table)[2]
    if sieve(us, vs, fac, B, n) == 0:
        return "no factors found, try larger limit, k primes base B"
    else:
        return f'a factor of {n} is {sieve(us, vs, fac, B, n)}'

In [180]:
print(NFS(2 ** 60 + 1, 15, 10))

a factor of 1152921504606846977 is 17
